In [3]:
from get_data import getData, createDriver, getAddress

ImportError: cannot import name 'createDriver' from 'get_data' (c:\Users\vrive\Documents\GitHub\vriveraq\ai-landscape-be\get_data.py)

In [6]:


ai_df = getData('https://www.ai4belgium.be/ai-landscape/')

In [7]:
addresses = []
for idx, row in ai_df.iterrows():
    address = getAddressfromSearch(row['Company Name'])
    addresses.append(address)
    print((idx,row['Company Name'], address))

(0, 'AE Projects', '')
(1, 'Agilytic', 'Rue F. Dubois 2, 1310 La Hulpe')
(2, 'Aptus', 'Meensesteenweg 449, 8501 Kortrijk')
(3, 'Arinti', '')
(4, 'Around Media', 'Adres: Kortrijksesteenweg 1127/0002, 9051, 9000 Gent')
(5, 'B12 Consulting', 'Boucle Odon Godart, 2, 1348, Ottignies-Louvain-la-Neuve')
(6, 'Blendr.io', '2017')
(7, 'BOBUP', 'Rue de Ransbeek 230, 1120 Bruxelles')
(8, 'Boltzmann', '20 februari 1844, Wenen, Oostenrijk')
(9, 'Brainjar', 'Gaston Geenslaan 11, 3001 Leuven')
(10, 'Brainstorm Consulting', '')
(11, 'Calculus', '')
(12, 'Chiveo', '')
(13, 'Comexis Partners', 'Avenue Pasteur 6, Building H, 1300 Wavre')
(14, 'CooperLink', 'Rue du Bois Saint-Jean 15/1, 4102 Seraing')
(15, 'Creax', 'Walle 113G, 8500 Kortrijk')
(16, 'CrossLang', 'Kerkstraat 106, 9050 Gent')
(17, 'D-Sight ', '2010')
(18, 'DART Consulting', 'Congresstraat 13, 1000 Brussel')
(19, 'Data Factory ', 'Sinter-Goedeleplein 5, 1000 Brussel')
(20, 'Data Minded', 'Vismarkt 17, 3000 Leuven')
(21, 'Dataa', 'Dieudonné Lef

In [8]:
ai_df['Address'] = addresses

In [9]:
ai_df.to_csv('data/AILandscape_From Name.csv', index=False)